In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("tutorial-4").getOrCreate()
sc    = spark.sparkContext      # get the context

# Set operations on RDDs

[Introduction to Spark with Python, by Jose A. Dianes](https://github.com/jadianes/spark-py-notebooks)

Spark supports many of the operations we have in mathematical sets, such as union and intersection, even when the RDDs themselves are not properly sets. It is important to note that these operations require that the RDDs being operated on are of the same type.  

Set operations are quite straightforward to understand as it work as expected. The only consideration comes from the fact that RDDs are not real sets, and therefore operations such as the union of RDDs doesn't remove duplicates. In this notebook we will have a brief look at `subtract`, `distinct`, and `cartesian`.       

In [2]:
data_file = "s3://gwu-bigdata/kdd1999/kddcup.data"
raw_data = sc.textFile(data_file)

## Getting attack interactions using `subtract`

For illustrative purposes, imagine we already have our RDD with non attack (normal) interactions from some previous analysis.   

In [3]:
normal_raw_data = raw_data.filter(lambda x: "normal." in x)

We can obtain attack interactions by subtracting normal ones from the original unfiltered RDD as follows.  

In [4]:
attack_raw_data = raw_data.subtract(normal_raw_data)

Let's do some counts to check our results.  

In [5]:
from time import time

# count all
t0 = time()
raw_data_count = raw_data.count()
tt = time() - t0
print "All count in {} secs".format(round(tt,3))

All count in 26.776 secs


In [6]:
# count normal
t0 = time()
normal_raw_data_count = normal_raw_data.count()
tt = time() - t0
print "Normal count in {} secs".format(round(tt,3))

Normal count in 12.199 secs


In [7]:
# count attacks
t0 = time()
attack_raw_data_count = attack_raw_data.count()
tt = time() - t0
print "Attack count in {} secs".format(round(tt,3))

Attack count in 38.646 secs


In [8]:
print "There are {} normal interactions and {} attacks, \
from a total of {} interactions".format(normal_raw_data_count,attack_raw_data_count,raw_data_count)

There are 972781 normal interactions and 3925650 attacks, from a total of 4898431 interactions


So now we have two RDDs, one with normal interactions and another one with attacks.  

## Protocol and service combinations using `cartesian`

We can compute the Cartesian product between two RDDs by using the `cartesian` transformation. It returns all possible pairs of elements between two RDDs. In our case we will use it to generate all the possible combinations between service and protocol in our network interactions.  

First of all we need to isolate each collection of values in two separate RDDs. For that we will use `distinct` on the CSV-parsed dataset. From the [dataset description](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names) we know that protocol is the second column and service is the third (tag is the last one and not the first as appears in the page).   

So first, let's get the protocols.  

In [9]:
csv_data = raw_data.map(lambda x: x.split(","))
protocols = csv_data.map(lambda x: x[1]).distinct()
protocols.collect()

[u'icmp', u'udp', u'tcp']

Now we do the same for services.  

In [10]:
services = csv_data.map(lambda x: x[2]).distinct()
services.collect()

[u'shell',
 u'domain_u',
 u'whois',
 u'gopher',
 u'printer',
 u'netbios_ssn',
 u'discard',
 u'ecr_i',
 u'http_2784',
 u'netbios_dgm',
 u'remote_job',
 u'daytime',
 u'other',
 u'vmnet',
 u'http_443',
 u'iso_tsap',
 u'auth',
 u'courier',
 u'netstat',
 u'smtp',
 u'eco_i',
 u'kshell',
 u'netbios_ns',
 u'ftp_data',
 u'imap4',
 u'http_8001',
 u'nnsp',
 u'efs',
 u'http',
 u'ctf',
 u'IRC',
 u'telnet',
 u'bgp',
 u'rje',
 u'link',
 u'X11',
 u'red_i',
 u'pm_dump',
 u'uucp_path',
 u'sunrpc',
 u'ftp',
 u'time',
 u'tftp_u',
 u'ldap',
 u'systat',
 u'login',
 u'aol',
 u'pop_3',
 u'uucp',
 u'name',
 u'exec',
 u'Z39_50',
 u'harvest',
 u'hostnames',
 u'ssh',
 u'nntp',
 u'pop_2',
 u'domain',
 u'sql_net',
 u'mtp',
 u'urp_i',
 u'private',
 u'klogin',
 u'finger',
 u'urh_i',
 u'supdup',
 u'tim_i',
 u'ntp_u',
 u'csnet_ns',
 u'echo']

A longer list in this case.

Now we can do the cartesian product.  

In [11]:
product = protocols.cartesian(services).collect()
print "There are {} combinations of protocol X service".format(len(product))

There are 210 combinations of protocol X service


Obviously, for such small RDDs doesn't really make sense to use Spark cartesian product. We could have perfectly collected the values after using `distinct` and do the cartesian product locally. Moreover, `distinct` and `cartesian` are expensive operations so they must be used with care when the operating datasets are large.    